In [1]:
from datasets import Dataset
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from transformers import AutoTokenizer, GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import zstandard as zstd
import chess.pgn
import io
import re
from torch.utils.data import DataLoader
import math
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import stockfish
import chess
import chess.engine

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# READ DATA

note to reader: this is where you can determine data size by choosing N when running read_first_n_games_as_strings

In [2]:
def format_pgn(text):

    # Remove metadata lines (anything within square brackets)
    text = re.sub(r'\[.*?\]', '', text)

    # Remove the result of the game (win, loss, or draw)
    text = re.sub(r'\s*(1-0|0-1|1/2-1/2)\s*', '', text)  # Remove game result

    # Remove excessive spaces and newlines
    text = ' '.join(text.split())

    # Remove `{ ... }` (comments or analysis)
    text = re.sub(r'\{.*?\}', '', text)

    # Normalize spaces 
    text = ' '.join(text.split())  

    # Define patterns for different components
    move_number_pattern = re.compile(r'(\d+\.)')  # Move numbers (e.g., "1.")
    piece_pattern = re.compile(r'([KQRBN])')  # Chess pieces (e.g., "N", "K")
    square_pattern = re.compile(r'([a-h][1-8])')  # Board squares (e4, d5, etc.)
    special_move_pattern = re.compile(r'(O-O|O-O-O|\+|#|x|=Q|=R|=B|=N)')  # Castling, check, capture, promotions
    
    # Ensure move numbers, pieces, and special moves are space-separated
    text = move_number_pattern.sub(r'\1 ', text)  # Move number spacing
    text = piece_pattern.sub(r'\1 ', text)  # Piece spacing
    text = special_move_pattern.sub(r' \1 ', text)  # Special moves spacing

    text += ' <EOS>'
    
    return ' '.join(text.split()) 


def read_first_n_games_as_strings(file_path, n=100):
    with open(file_path, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        decompressed = dctx.stream_reader(f)
        pgn_text = io.TextIOWrapper(decompressed, encoding='utf-8')

        games = []
        for _ in range(n):
            game = chess.pgn.read_game(pgn_text)
            if game is None:
                break  # Stop if no more games are available
            
            # Convert game to string
            game_str = io.StringIO()
            game.accept(chess.pgn.StringExporter())  # Corrected line
            games.append(format_pgn(str(game)))

    return games


# PREP DATA FOR TOKENIZER

In [ ]:
# Replace with your file path
file_path = "INSERT_PATH_HERE"
# Replace with number of games
number_of_games = 100000
games_as_strings = read_first_n_games_as_strings(file_path, n=number_of_games)

# Print the first game's PGN as a string
print(games_as_strings[0])

# Convert to Hugging Face Dataset format
dataset = Dataset.from_dict({"text": games_as_strings})

train_data, val_data = train_test_split(dataset["text"], test_size=0.2, random_state=42)

train_dataset = Dataset.from_dict({"text": train_data})
val_dataset = Dataset.from_dict({"text": val_data})

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})


1. d4 1. .. N f6 2. f3 2. .. d5 3. c4 3. .. c6 4. e3 4. .. g6 5. N c3 5. .. B g7 6. B d3 6. .. O-O 7. N ge2 7. .. N a6 8. a3 8. .. N c7 9. b4 9. .. R e8 10. O-O 10. .. B d7 11. c5 11. .. e5 12. h3 12. .. e x d4 13. N x d4 13. .. N e6 14. R b1 14. .. N x d4 15. e x d4 15. .. N h5 16. N e2 16. .. Q f6 17. B c2 17. .. B f5 18. B x f5 18. .. Q x f5 19. B b2 19. .. N f4 20. N x f4 20. .. Q x f4 21. Q c2 21. .. R e3 22. B c1 22. .. B x d4 23. B x e3 23. .. B x e3 + 24. K h1 24. .. h5 25. Q e2 25. .. d4 26. R be1 26. .. R e8 27. Q d3 27. .. R e6 28. R e2 28. .. Q g3 29. R x e3 29. .. d x e3 30. Q d8 + 30. .. K g7 31. Q d3 31. .. e2 32. R g1 32. .. e1 =Q 33. R x e1 33. .. R x e1 + <EOS>


# TRAIN TOKENIZER

note to reader: if experimenting with model size, adjust tokenizer name accordingly below

In [5]:
# note to reader: if experimenting with model size, adjust tokenizer name accordingly below
tokenizer_name = "gpt2"
old_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Define special tokens
special_tokens = {
    "eos_token": "<EOS>",
    "pad_token": "[PAD]"
}

# Resize the tokenizer with the new EOS token
old_tokenizer.add_special_tokens(special_tokens)

# Train a new tokenizer using the updated old_tokenizer
tokenizer = old_tokenizer.train_new_from_iterator(dataset['train'], 120)

# TOKENIZE DATA

In [6]:
# Tokenize Function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128,
        return_special_tokens_mask=True
    )


# Tokenize Dataset
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

train_subset = tokenized_datasets["train"]
eval_subset = tokenized_datasets["validation"]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Map: 100%|██████████| 20000/20000 [00:03<00:00, 5152.78 examples/s]


# SET UP AND CONFIGURE MODEL

In [ ]:
# Create GPT model configuration
# note to reader: you can modify configuration to see how it affects chess performance
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,  # Match tokenizer's vocab size
    n_positions=128,
    n_embd=768,
    n_layer=12,
    n_head=12
)

# Initialize model
model = GPT2LMHeadModel(config)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False 
)

training_args = TrainingArguments(
    output_dir="./gpt-chess",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    warmup_steps=500,
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()



/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/vn/ldnd29fd4v1dzjdkm158113w0000gn/T/ipykernel_5503/3667286928.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 10/30000 [00:03<2:42:54,  3.07it/s]

{'loss': 5.3283, 'grad_norm': 70.04059600830078, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/30000 [00:06<2:39:13,  3.14it/s]

{'loss': 3.6905, 'grad_norm': 16.219356536865234, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 30/30000 [00:09<2:39:28,  3.13it/s]

...

 20%|█▉        | 5930/30000 [32:37<2:07:18,  3.15it/s]

{'loss': 0.2658, 'grad_norm': 0.5293242335319519, 'learning_rate': 4.0796610169491526e-05, 'epoch': 0.59}


 20%|█▉        | 5940/30000 [32:40<2:07:09,  3.15it/s]

{'loss': 0.293, 'grad_norm': 0.6221745014190674, 'learning_rate': 4.0779661016949155e-05, 'epoch': 0.59}


 20%|█▉        | 5946/30000 [32:42<2:07:24,  3.15it/s]

# EVALUATE MODEL

In [ ]:
import torch
device = torch.device("cpu")
model.to(device)

In [ ]:
class ChessSimulator:
    """
    ChessSimulator simulates a chess game between the pre-trained model and the Stockfish chess engine.
    """
    def __init__(self, model, tokenizer, stockfish_filepath):
        """
        Initialize a chess simulation between the pretrained the Stockfish chess engine.
        This involves initializing a Board state from py-chess to keep track of moves made 
        and an engine instance from Stockfish that can respond accordingly.
        Additionally, this keeps track of the move number and whether or not it is the model's turn in 
        order to construct the prompt the model is expecting in order to generate the next move.
        Note: there are two sequences in a move number; one for white and for black. 
        White's move number is formatted as '{move_number}. ' and black's move number is formatted as '{move_number}. ..'.

        
        Parameters:
            - model (transformers.PreTrainedModel): A Hugging Face transformer model
            such as BERT, GPT-2, T5, etc., pretrained on PGN formatted data.
            - tokenizer (transformers.PreTrainedTokenizer): The corresponding tokenizer
            for the model, used to encode PGN formatted games.
            stockfish_filepath (str, optional): Filepath to Stockfish binary.
        """
        self.model = model
        self.tokenizer = tokenizer
        self.board = chess.Board()
        self.engine = chess.engine.SimpleEngine.popen_uci(stockfish_filepath)
        self.prompt = ''
        self.move_number = 1
        self.modelTurn = True
        self.currentMove = None
    
    def simulateGame(self):
        """
        This function simulates a chess game where the model starts and the engine responds. 
        Each move is processed accordingly to be added into the board and again to update the prompt for the model.
        It returns whether or not the model failed, won/drew a tie, or lost.
        """
        while not self.board.is_checkmate():
            try:
                if self.modelTurn:
                    self.generateModelMove()
                    print(self.prompt)
                else:
                    self.generateChessMove()
                    self.move_number += 1
                self.modelTurn = not self.modelTurn

            except:
                return -1 # signify model error or hallucination
            
        self.engine.quit()

        if self.board.is_checkmate():
            if self.board.turn == chess.WHITE:
                return 0
            else: # counts draws as a win
                return 1
            
    def extract_white_move(self,text):
        """
        Takes in model response and parses relevant move as it pertains to model number.

        Parameters:
            - text (str): model response to prompt
        """
        # Meant to extract white move from generated model string, based on the current move number
        # Normalize spaces
        cleaned_text = " ".join(text.split())

        # Define start and end markers
        start_marker = f"{self.move_number}."
        end_marker = f"{self.move_number}. .."

        # Find the position of the move number
        start_index = cleaned_text.find(start_marker)
        if start_index == -1:
            return None  # Move not found

        # Find the position of the next black move
        end_index = cleaned_text.find(end_marker, start_index)
        
        # Extract substring
        if end_index != -1:
            move_text = cleaned_text[start_index + len(start_marker):end_index].strip()
        else:
            # If no black move is found, take everything after the move number
            move_text = cleaned_text[start_index + len(start_marker):].strip().split()[0]

        return move_text

    def parseModelMove(self, move):
        """
        Takes in model response, extracts relevant to current move number.
        Processes parsed response accordingly to get pushed to chess board by converting to SAN format.
        Processes parsed response accordingly for model prompt by separating move, special pieces, special moves, etc by white spaces
        as is consistent with the preprocessing/tokenization process.

        Parameters:
            - text (str): model response to prompt
        """
        # Extract move using regex
        move = self.extract_white_move(move)
        # Need to strip whitespace, this should accept special pieces and special moves
        # as they're in "algebraic notation"
        self.board.push_san(move.replace(' ',''))
        self.prompt += move

    
    def parseChessMove(self, move):
        """
        Pushes engine response to chess board. Parses engine response so it's consistent with format model is expecting for addition to the prompt by
        separating out special pieces, special moves, moves by white spaces.

        Parameters:
            - move (str): Response returned by chess engine
        """
        # converts move in UCI notation to SAN notation, that model expects
        self.prompt += f' {self.move_number}. .. ' # note: black's move number is always denoted afterwards with a '. ..' as per model standards
        san_move = self.board.san(move)
        # Define patterns for different components
        move_number_pattern = re.compile(r'(\d+\.)')  # Move numbers (e.g., "1.")
        piece_pattern = re.compile(r'([KQRBN])')  # Chess pieces (e.g., "N", "K")
        square_pattern = re.compile(r'([a-h][1-8])')  # Board squares (e4, d5, etc.)
        special_move_pattern = re.compile(r'(O-O|O-O-O|\+|#|x|=Q|=R|=B|=N)')  # Castling, check, capture, promotions
            
        # Ensure move numbers, pieces, and special moves are space-separated
        san_move = move_number_pattern.sub(r'\1 ', san_move)  # Move number spacing
        san_move = piece_pattern.sub(r'\1 ', san_move)  # Piece spacing
        san_move = special_move_pattern.sub(r' \1 ', san_move)  # Special moves spacing
        self.prompt += san_move +' '


    def generateModelMove(self):
        """
        Generates model response and updates prompt.
        """
        self.prompt += f'{self.move_number}. ' # note: white's move number is always denoted afterwards with a singular '.' as per model standards
        inputs = tokenizer(self.prompt, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=15, num_return_sequences=1) # generate lots of tokens, and parse for information relevant to current move only
        move = tokenizer.decode(outputs[0], skip_special_tokens=True)
        self.currentMove = self.parseModelMove(move)


    def generateChessMove(self):
        """
        Calls chess engine's API to respond to board, parse the move to update the model prompt, and then get pushed to the board.
        """
        result = self.engine.play(self.board, chess.engine.Limit(time=2.0))  # Time limit for the move
        # move needs to parsed into SAN notation before being pushed to board
        self.parseChessMove(result.move)
        self.board.push(result.move)
        


In [ ]:

game_outcomes = {0: 0, -1: 0, 1: 0}

for _ in range(100):
    game = ChessSimulator(model, tokenizer, "PATH_TO_STOCKFISH_BINARY")
    game_outcomes[game.simulateGame()] += 1

game_outcomes